In [ ]:
!pip install osmnx folium networkx geopandas rtree


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 14.2 MB/s eta 0:00:00


In [ ]:
df_여행객 = pd.read_csv("/content/drive/MyDrive/V3_관광지 카테고리/V3_동반자여행객.csv")       # TRAVEL_ID, GENDER, AGE_GRP, COMPANION_TYPE
df_여행 = pd.read_csv("/content/drive/MyDrive/V3_관광지 카테고리/V3_여행.csv")              # TRAVEL_ID, POI_ID, satisfaction, revisit_flag, recommend_flag
df_관광지  = pd.read_csv("/content/drive/MyDrive/V3_관광지 카테고리/V3_관광지Data_권역추가.csv")   # POI_ID, 권역, 소분류, latitude, longitude

In [ ]:
df_여행객

,Unnamed: 0,TRAVEL_ID,REL_CD,REL_CD_NEW,동반자 관계,TRAVELER_ID,GENDER,AGE_GRP,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,성별_나이대_동반자관계
0,0,e_e000295,7,2.0,친구,e000295,남,20,3,1.0,2.0,남_20_친구
1,1,e_e000295,7,2.0,친구,e000295,남,20,3,1.0,2.0,남_20_친구
2,2,e_e000295,7,2.0,친구,e000295,남,20,3,1.0,2.0,남_20_친구
3,3,e_e000295,7,2.0,친구,e000295,남,20,3,1.0,2.0,남_20_친구
4,4,e_e000295,7,2.0,친구,e000295,남,20,3,1.0,2.0,남_20_친구
...,...,...,...,...,...,...,...,...,...,...,...,...
3717,3717,h_h006755,8,3.0,연인,h006755,남,30,7,1.0,2.0,남_30_연인
3718,3718,h_h006766,7,2.0,친구,h006766,남,50,5,8.0,1.0,남_50_친구
3719,3719,h_h006870,2,1.0,가족,h006870,여,50,10,1.0,8.0,여_50_가족
3720,3720,h_h006872,1,3.0,연인,h006872,여,30,8,5.0,1.0,여_30_연인


In [ ]:
df_여행

,Unnamed: 0,TRAVEL_ID,TRAVEL_NM,TRAVELER_ID,TRAVEL_MISSION_CHECK,TM1,TM2,TM3
0,0,e_e000295,E03,e000295,7;1;22,7,1,0
1,1,e_e000305,H02,e000305,22;21;27,0,0,0
2,2,e_e000335,H02,e000335,1;6;26,6,1,0
3,3,e_e000365,H02,e000365,5;11;12,12,11,5
4,4,e_e000401,H02,e000401,1;23;21,1,0,0
...,...,...,...,...,...,...,...,...
2555,2555,h_h006755,H01,h006755,22;26;3,3,0,0
2556,2556,h_h006766,H01,h006766,6;9;22,9,6,0
2557,2557,h_h006870,H01,h006870,6;22;5,6,5,0
2558,2558,h_h006872,H01,h006872,11;9;3,11,9,3


In [ ]:
df_관광지

In [ ]:
import pandas as pd
import numpy as np
import random
from math import radians, cos, sin, asin, sqrt
import osmnx as ox
import networkx as nx
import folium

In [ ]:

# -------------------------------
# 1. 데이터 로드 및 전처리
# -------------------------------

def load_traveler_profiles(path_profiles, path_companions):
    df1 = pd.read_csv(path_profiles, dtype=str)      # V3_여행 (1).csv
    df2 = pd.read_csv(path_companions, dtype=str)    # V3_동반자여행객 (1).csv

    # TRAVEL_ID 기준으로 합치기
    df = pd.merge(df1, df2, on="TRAVEL_ID", how="inner")

    # 이미 "남_30_가족" 형태로 된 컬럼을 그대로 CATEGORY로 사용
    df["CATEGORY"] = df["성별_나이대_동반자관계"]

    return df[["TRAVEL_ID","TRAVELER_ID","CATEGORY"]]


In [ ]:
def load_poi_data(path_poi):
    """
    V3_관광지Data_권역추가.csv 를 불러와
    DGSTFN, REVISIT_INTENTION, RCMDTN_INTENTION, REVISIT_YN,
    Y_COORD, X_COORD 를 숫자형으로 변환
    """
    df = pd.read_csv("/content/drive/MyDrive/V3_관광지 카테고리/V3_관광지Data_권역추가.csv", dtype=str)

    # 숫자형 변환
    df["DGSTFN"]            = df["DGSTFN"].astype(float)             # 만족도
    df["REVISIT_INTENTION"] = df["REVISIT_INTENTION"].astype(float)  # 재방문의향
    df["RCMDTN_INTENTION"]  = df["RCMDTN_INTENTION"].astype(float)   # 추천의향
    df["REVISIT_YN"]        = df["REVISIT_YN"].map({"Y":1, "N":0}).astype(float)

    # 좌표
    df["LAT"] = df["Y_COORD"].astype(float)
    df["LON"] = df["X_COORD"].astype(float)

    return df

In [ ]:
# 만족도, 재방문 의향, 추천 의향을 Total Score로 변환
df_관광지['Score'] = df_관광지['DGSTFN'] * 0.4+ df_관광지['REVISIT_INTENTION'] * 0.3 + df_관광지['RCMDTN_INTENTION']* 0.3


In [ ]:
df_관광지

In [ ]:
def normalize_scores(df):
    """
    DGSTFN, REVISIT_INTENTION, RCMDTN_INTENTION, REVISIT_YN 을 0-1 정규화
    """
    for col in ["DGSTFN","REVISIT_INTENTION","RCMDTN_INTENTION","REVISIT_YN"]:
        mn, mx = df[col].min(), df[col].max()
        df[col + "_NORM"] = (df[col] - mn) / (mx - mn)
    return df

In [ ]:
# -------------------------------
# 2. 카테고리 기반 지역 필터링
# -------------------------------

def top_regions_by_category(df_trav, df_poi, top_k=5):
    """
    각 CATEGORY 별로 POI가 위치한 권역의 방문 빈도를 정규화한 뒤
    상위 top_k 개 권역을 반환
    """
    merged = pd.merge(
        df_trav[["TRAVEL_ID","CATEGORY"]],
        df_poi[["VISIT_AREA_ID","권역"]],
        left_on="TRAVEL_ID", right_on="VISIT_AREA_ID",
        how="inner"
    )
    freq = merged.groupby(["CATEGORY","권역"]).size().reset_index(name="cnt")

    def normalize(group):
        group["cnt_N"] = (group.cnt - group.cnt.min()) / (group.cnt.max() - group.cnt.min())
        return group

    normed = freq.groupby("CATEGORY").apply(normalize)
    top = normed.sort_values(["CATEGORY","cnt_N"], ascending=[True,False]) \
                .groupby("CATEGORY").head(top_k)
    return top  # columns: CATEGORY, 권역, cnt, cnt_N

In [ ]:
# -------------------------------
# 3. 만족도 기반 스코어링
# -------------------------------

def compute_composite_score(df, weights=(0.4,0.3,0.3)):
    """
    논문식 가중치로 종합 만족도 SCORE 계산
    """
    w1,w2,w3,w4 = weights
    df["SCORE"] = (
        w1*df["DGSTFN_NORM"] +
        w2*df["REVISIT_INTENTION_NORM"] +
        w3*df["RCMDTN_INTENTION_NORM"]
    )
    return df

In [ ]:
# -------------------------------
# 4. 거리 기반 필터링
# -------------------------------

def haversine(lon1, lat1, lon2, lat2):
    """
    두 위경도 사이의 거리를 km 단위로 계산
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1,lat1,lon2,lat2])
    dlon = lon2-lon1; dlat = lat2-lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    km = 2*asin(sqrt(a))*6371
    return km

In [ ]:
def filter_by_distance(df, max_dist_km=10):
    """
    후보 POI 간 최소 거리가 max_dist_km 이하인 것만 남김
    """
    coords = df[["LON","LAT"]].values
    keep = []
    for i,(x1,y1) in enumerate(coords):
        dists = [haversine(x1,y1,x2,y2)
                 for j,(x2,y2) in enumerate(coords) if i!=j]
        keep.append(bool(dists and min(dists) <= max_dist_km))
    return df[keep]


In [ ]:
# -------------------------------
# 5. 다양성 확보 (탐색적 추천)
# -------------------------------

def diversify_candidates(df, fraction=0.2):
    """
    최종 후보 중 fraction 비율만큼
    랜덤 하위 POI로 대체하여 다양성 확보
    """
    n = len(df)
    num_replace = int(n * fraction)
    low_pool = df.sample(min(len(df), num_replace*5))
    to_replace = df.sample(num_replace).index
    df.loc[to_replace, ["VISIT_AREA_ID","POI_NM","LAT","LON"]] = (
        low_pool.sample(num_replace)[["VISIT_AREA_ID","POI_NM","LAT","LON"]].values
    )
    return df

In [ ]:
# -------------------------------
# 6. 최단 경로 계산 및 시각화
# -------------------------------

def compute_shortest_path(poi_df, start_point=(33.5063,126.4934)):
    """
    제주 공항(start_point)부터 POI 리스트를 순회하는 최단 경로 노드 리스트 구함
    """
    G = ox.graph_from_place("Jeju Island, South Korea", network_type="drive")
    orig_node = ox.get_nearest_node(G, (start_point[0], start_point[1]))
    path_nodes = [orig_node]
    for _, row in poi_df.iterrows():
        node = ox.get_nearest_node(G, (row["LAT"], row["LON"]))
        path_nodes.append(node)

    full_path = []
    for u, v in zip(path_nodes[:-1], path_nodes[1:]):
        segment = nx.shortest_path(G, u, v, weight="length")
        full_path.extend(segment)
    return G, full_path

In [ ]:
def plot_route(G, path_nodes, output_html="route_map.html"):
    """
    Folium 지도에 최단 경로를 그려 HTML로 저장
    """
    m = folium.Map(location=[33.5063,126.4934], zoom_start=10)
    coords = [(G.nodes[n]["y"], G.nodes[n]["x"]) for n in path_nodes]
    folium.PolyLine(coords, weight=5, opacity=0.7).add_to(m)
    m.save(output_html)
    print(f"추천 경로 지도를 {output_html} 에 저장했습니다.")


In [ ]:
if __name__ == "__main__":
    # 1) 데이터 로드
    df_trav = load_traveler_profiles("/content/drive/MyDrive/V3_관광지 카테고리/V3_여행.csv", "/content/drive/MyDrive/V3_관광지 카테고리/V3_동반자여행객.csv")
    df_poi  = load_poi_data("/content/drive/MyDrive/V3_관광지 카테고리/V3_관광지Data_권역추가.csv")
    df_poi  = normalize_scores(df_poi)

    # 2) 카테고리 → 권역 필터링
    top_regs = top_regions_by_category(df_trav, df_poi, top_k=5)
    user_cat = df_trav["CATEGORY"].iloc[0]  # 예: 첫 번째 사용자 카테고리
    regions  = top_regs[top_regs["CATEGORY"]==user_cat]["권역"].tolist()

    # 3) 점수 기반 후보 추출
    df_poi = compute_composite_score(df_poi)
    candidates = df_poi[df_poi["권역"].isin(regions)].nlargest(20, "SCORE")

    # 4) 거리 필터링
    candidates = filter_by_distance(candidates, max_dist_km=10)

    # 5) 다양성 확보
    candidates = diversify_candidates(candidates, fraction=0.2)

    # 6) 최단 경로 계산 및 시각화
    G, path = compute_shortest_path(candidates)
    plot_route(G, path, output_html="route_map.html")